In [15]:
import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment
import os

In [2]:
ws = Workspace.from_config()

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
for exp in Experiment.list(ws):
    print(exp.name)

dataset_profile
loan_model_with_designer
new_test_experiment
new_experiment_1
mslearn-diabetes
mslearn-diabetes_1
mslearn-diabetes-mlflow
mslearn-train-diabetes
mslearn-diabetes-pipeline
mslearn-diabetes-batch


In [4]:
experiment = Experiment(workspace=ws, name='mslearn-diabetes')

In [7]:
list(experiment.get_runs())

[Run(Experiment: mslearn-diabetes,
 Id: mslearn-diabetes_1671652660_47e6abb7,
 Type: azureml.scriptrun,
 Status: Completed),
 Run(Experiment: mslearn-diabetes,
 Id: mslearn-diabetes_1671652538_818d93b6,
 Type: azureml.scriptrun,
 Status: Failed),
 Run(Experiment: mslearn-diabetes,
 Id: mslearn-diabetes_1671652367_d5470828,
 Type: azureml.scriptrun,
 Status: Failed),
 Run(Experiment: mslearn-diabetes,
 Id: mslearn-diabetes_1671651671_3430dd27,
 Type: azureml.scriptrun,
 Status: Failed),
 Run(Experiment: mslearn-diabetes,
 Id: cbe76cc6-0ee5-4599-9181-9e466830e428,
 Type: None,
 Status: Completed),
 Run(Experiment: mslearn-diabetes,
 Id: b8ba110f-ba95-4f48-a901-f66a6ae43dd6,
 Type: None,
 Status: Completed),
 Run(Experiment: mslearn-diabetes,
 Id: 479f2325-3cd7-46a7-b07a-269b72616438,
 Type: None,
 Status: Canceled)]

In [13]:
run.get(workspace=ws, run_id='mslearn-diabetes_1671652538_818d93b6')

Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-diabetes,mslearn-diabetes_1671652538_818d93b6,azureml.scriptrun,Failed,Link to Azure Machine Learning studio,Link to Documentation


In [8]:
run = experiment.start_logging()

In [9]:
print(run.get_status())

Running


In [11]:
print(run.get_details())

{'runId': 'c7d93f97-c468-4370-8216-e8ac44abfa69', 'target': 'local', 'status': 'Running', 'startTimeUtc': '2023-01-10T15:29:14.084568Z', 'services': {}, 'properties': {'azureml.git.repository_uri': 'https://github.com/wliu40/mslearn-dp100.git', 'mlflow.source.git.repoURL': 'https://github.com/wliu40/mslearn-dp100.git', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50', 'mlflow.source.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50', 'azureml.git.dirty': 'True', 'ContentSnapshotId': 'f345f26d-b5d9-488e-8eb8-2045a561c7cd'}, 'inputDatasets': [], 'outputDatasets': [], 'logFiles': {}, 'submittedBy': 'Wei Liu'}


Check this link for the API docs:
https://learn.microsoft.com/en-us/python/api/azureml-core/azureml.core.run.run?view=azure-ml-py

Some more examples:
https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-track-monitor-analyze-runs?tabs=python


- run.log(name='accuracy', value=0.99)

- run.log_image(name='distribution.png', plot=fig)

- run.log_row(col, stat=keys[index], value = values[index])

- run.log_list()

- run.get_metrics()

- run.get_file_names()

- run.get_details_with_logs()

- run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

- run.download_files(prefix='outputs', output_directory='downloaded_files') # files will be saved to downloaded-files\outputs

To add searchable metadata to your runs, use the add_properties() method. For example, the following code adds the "author" property to the run:
- run.add_properties({"author":"azureml-user"}
- run.get_properties()

Unlike properties, tags are mutable. To add searchable and meaningful information for consumers of your experiment, use the tag() method.

- local_run.tag("quality", "great run")
- print(local_run.get_tags())


- list(exp.get_runs(properties={"author":"azureml-user"},tags={"quality":"fantastic run"}))

In [16]:
# Create a folder for the experiment files
folder_name = 'playground'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

In [29]:
%%writefile $folder_name/data_input.py
from azureml.core import Run
import pandas as pd
import os

# Get the experiment run context
run = Run.get_context()

data = pd.DataFrame({'a':[1,2,4,5,6], 'b':[3,5,6,7,8]})

run.log(name="df_size", value=len(data))

run.tag('quality', 'good')

run.add_properties({"author": "wei"})

# Complete the run
run.complete()


Overwriting playground/data_input.py


In [30]:
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration
from azureml.widgets import RunDetails

environment = Environment.get(workspace=ws, name='experiment_env')

script_config = ScriptRunConfig(source_directory=folder_name,
                               script='data_input.py',
                               environment=environment,
                               docker_runtime_config=DockerConfiguration(use_docker=True),
                               compute_target='wliu08281'
                               )

experiment = Experiment(workspace=ws, name='playground')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()


{'runId': 'playground_1673397068_2915746b',
 'target': 'wliu08281',
 'status': 'Completed',
 'startTimeUtc': '2023-01-11T00:31:21.942132Z',
 'endTimeUtc': '2023-01-11T00:33:38.504101Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '146e8a84-368f-4768-8c6a-a7afe3497b64',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.git.repository_uri': 'https://github.com/wliu40/mslearn-dp100.git',
  'mlflow.source.git.repoURL': 'https://github.com/wliu40/mslearn-dp100.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50',
  'mlflow.source.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50',
  'azureml.git.dirty': 'True',
  'author': 'wei'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'data_input.py',
  'command': '',
  'useAbsolutePath': False,
  'ar

In [31]:
for i,j in run.get_metrics().items():
    print(i, j)

df_size 5


In [38]:
list(experiment.get_runs(tags={'quality': 'good'}))[0].get_details()

{'runId': 'playground_1673397068_2915746b',
 'target': 'wliu08281',
 'status': 'Completed',
 'startTimeUtc': '2023-01-11T00:31:21.942132Z',
 'endTimeUtc': '2023-01-11T00:33:38.504101Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '146e8a84-368f-4768-8c6a-a7afe3497b64',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json',
  'azureml.git.repository_uri': 'https://github.com/wliu40/mslearn-dp100.git',
  'mlflow.source.git.repoURL': 'https://github.com/wliu40/mslearn-dp100.git',
  'azureml.git.branch': 'main',
  'mlflow.source.git.branch': 'main',
  'azureml.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50',
  'mlflow.source.git.commit': 'b158bead3315d2909d5d12e33621494bb5b49c50',
  'azureml.git.dirty': 'True',
  'author': 'wei'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'data_input.py',
  'command': '',
  'useAbsolutePath': False,
  'ar